# Setup

In [2]:
import pandas as pd

# Merge and filter data
-  Read all the sensor data
- Filter for only data from sensors present in our model
- Propperly read the timestamps
- Aggregate all the processed data into a new dataframe

In [5]:
# File names and relative paths
files = ['./Dataset/2013AEDL.csv', './Dataset/1S2014AEDL.csv', './Dataset/2S2014AEDL.csv', './Dataset/1P2015AEDL.csv', './Dataset/2P2015AEDL.csv']

# Define the list of ids from the sensors used in our model
target_ids = [121726,121727,121731,121732,121733,121734,121735,121736,121741,121742,121754,121755,121756]

# Filter the DataFrame: we want to keep only rows where 'EQUIPMENTID' matches the sensor IDs we use
col = 'EQUIPMENTID'

# Create dataframe to concatenate all the useful information
filtered_df = pd.DataFrame()

for file in files:
    df = pd.read_csv(file, parse_dates=["AGG_PERIOD_START"])

    # Convert data types before filtering
    df["AGG_PERIOD_START"] = pd.to_datetime(df["AGG_PERIOD_START"])
    df["LANE_BUNDLE_DIRECTION"] = df["LANE_BUNDLE_DIRECTION"].astype("string")
    
    # Filter for only rows with matching sensor ids
    matching_df = df[df[col].isin(target_ids)]
    
    filtered_df = pd.concat([filtered_df, matching_df], ignore_index=True)

    print(f"Original rows: {len(df)}")
    print(f"Filtered rows: {len(matching_df)}")

filtered_df.info()

Original rows: 3890630
Filtered rows: 2063480
Original rows: 2251668
Filtered rows: 1153163
Original rows: 2745968
Filtered rows: 1312094
Original rows: 1315265
Filtered rows: 648009
Original rows: 3881466
Filtered rows: 1917689
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7094435 entries, 0 to 7094434
Data columns (total 20 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   AGGREGATE_BY_LANE_BUNDLEID  int64         
 1   AGG_ID                      int64         
 2   EQUIPMENTID                 int64         
 3   AGG_PERIOD_START            datetime64[ns]
 4   AGG_PERIOD_LEN_MINS         int64         
 5   NR_LANES                    int64         
 6   LANE_BUNDLE_DIRECTION       string        
 7   TOTAL_VOLUME                int64         
 8   AVG_SPEED_ARITHMETIC        float64       
 9   AVG_SPEED_HARMONIC          float64       
 10  AVG_LENGTH                  float64       
 11  AVG_SPACING                 f

# Transform data for MAB

## Eliminate useless columns

In [6]:
simple_df = filtered_df.drop(columns=["AGGREGATE_BY_LANE_BUNDLEID","AGG_ID","AGG_PERIOD_LEN_MINS","NR_LANES","AVG_LENGTH","OCCUPANCY","VOLUME_CLASSE_A","VOLUME_CLASSE_B","VOLUME_CLASSE_C","VOLUME_CLASSE_D","VOLUME_CLASSE_0","AXLE_CLASS_VOLUMES","AVG_SPEED_ARITHMETIC","AVG_SPEED_HARMONIC","AVG_SPACING","LIGHT_VEHICLE_RATE"])

## Create tags for arms
### (gather all data from the same sensor, the same time interval and the same direction)

In [7]:
# Map data into arm keys
# Arm = (time_bin, sensor_id, direction)

# Minutes since midnight
simple_df["min_since_midnight"] = (simple_df["AGG_PERIOD_START"].dt.hour*60 + simple_df["AGG_PERIOD_START"].dt.minute)

# 5‑minute bins: [0..287]
simple_df["simple_time_bin"] = (simple_df["min_since_midnight"] // 5).astype("int16")
# This maps 00:00–00:04 → bin 0, 00:05–00:09 → bin 1, …, 23:55–23:59 → bin 287, across all days.

# 30‑minute bins: [0..47]
simple_df["half_time_bin"] = (simple_df["min_since_midnight"] // 30).astype("int8")

# 1 hour bins: [0..23]
simple_df["one_time_bin"] = (simple_df["min_since_midnight"] // 60).astype("int8")

# Define arm identifiers (5-min bin, sensor, direction) as the arm key
simple_df["simple_arm"] = list(
    zip(
        simple_df["simple_time_bin"],
        simple_df["EQUIPMENTID"],
        simple_df["LANE_BUNDLE_DIRECTION"]
    )
)

# Define arm identifiers: (30‑min bin, sensor, direction)
simple_df["half_arm"] = list(
    zip(
        simple_df["half_time_bin"],
        simple_df["EQUIPMENTID"],
        simple_df["LANE_BUNDLE_DIRECTION"]
    )
)

# Define arm identifiers: (1hr bin, sensor, direction)
simple_df["one_arm"] = list(
    zip(
        simple_df["one_time_bin"],
        simple_df["EQUIPMENTID"],
        simple_df["LANE_BUNDLE_DIRECTION"]
    )
)

In [11]:
print("Ammount of arms per time interval aggregation")
simple = simple_df["simple_arm"].nunique()
print(f"5 minutes: {simple}")
half = simple_df["half_arm"].nunique()
print(f"30 minutes: {half}")
one = simple_df["one_arm"].nunique()
print(f"1 hour: {one}")

Ammount of arms per time interval aggregation
5 minutes: 7488
30 minutes: 1248
1 hour: 624


## Final Preprocessed Data

In [12]:
simple_df

,EQUIPMENTID,AGG_PERIOD_START,LANE_BUNDLE_DIRECTION,TOTAL_VOLUME,min_since_midnight,simple_time_bin,half_time_bin,one_time_bin,simple_arm,half_arm,one_arm
0,121726,2013-04-12 01:05:00,C,43,65,13,2,1,"(13, 121726, C)","(2, 121726, C)","(1, 121726, C)"
1,121726,2013-04-12 01:05:00,D,60,65,13,2,1,"(13, 121726, D)","(2, 121726, D)","(1, 121726, D)"
2,121726,2013-04-12 01:10:00,C,48,70,14,2,1,"(14, 121726, C)","(2, 121726, C)","(1, 121726, C)"
3,121726,2013-04-12 01:10:00,D,64,70,14,2,1,"(14, 121726, D)","(2, 121726, D)","(1, 121726, D)"
4,121726,2013-04-12 01:15:00,C,37,75,15,2,1,"(15, 121726, C)","(2, 121726, C)","(1, 121726, C)"
...,...,...,...,...,...,...,...,...,...,...,...
7094430,121733,2015-12-31 15:55:00,D,365,955,191,31,15,"(191, 121733, D)","(31, 121733, D)","(15, 121733, D)"
7094431,121741,2015-12-31 15:55:00,C,297,955,191,31,15,"(191, 121741, C)","(31, 121741, C)","(15, 121741, C)"
7094432,121741,2015-12-31 15:55:00,D,359,955,191,31,15,"(191, 121741, D)","(31, 121741, D)","(15, 121741, D)"
7094433,121741,2015-12-31 16:00:00,C,266,960,192,32,16,"(192, 121741, C)","(32, 121741, C)","(16, 121741, C)"


In [13]:
simple_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7094435 entries, 0 to 7094434
Data columns (total 11 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   EQUIPMENTID            int64         
 1   AGG_PERIOD_START       datetime64[ns]
 2   LANE_BUNDLE_DIRECTION  string        
 3   TOTAL_VOLUME           int64         
 4   min_since_midnight     int32         
 5   simple_time_bin        int16         
 6   half_time_bin          int8          
 7   one_time_bin           int8          
 8   simple_arm             object        
 9   half_arm               object        
 10  one_arm                object        
dtypes: datetime64[ns](1), int16(1), int32(1), int64(2), int8(2), object(3), string(1)
memory usage: 433.0+ MB


In [14]:
simple_df.describe()

,EQUIPMENTID,AGG_PERIOD_START,TOTAL_VOLUME,min_since_midnight,simple_time_bin,half_time_bin,one_time_bin
count,7.094435e+06,7094435,7.094435e+06,7.094435e+06,7.094435e+06,7.094435e+06,7.094435e+06
mean,1.217385e+05,2014-08-08 19:08:26.350710016,1.807496e+02,7.172668e+02,1.434534e+02,2.349221e+01,1.149610e+01
min,1.217260e+05,2013-03-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.217320e+05,2013-11-20 00:00:00,5.800000e+01,3.550000e+02,7.100000e+01,1.100000e+01,5.000000e+00
50%,1.217350e+05,2014-08-18 02:20:00,1.610000e+02,7.150000e+02,1.430000e+02,2.300000e+01,1.100000e+01
75%,1.217420e+05,2015-04-21 20:55:00,2.840000e+02,1.080000e+03,2.160000e+02,3.600000e+01,1.800000e+01
max,1.217560e+05,2015-12-31 23:55:00,8.750000e+02,1.435000e+03,2.870000e+02,4.700000e+01,2.300000e+01
std,9.933554e+00,NaN,1.325153e+02,4.158539e+02,8.317078e+01,1.385887e+01,6.924901e+00


# Train/Test Split

In [15]:
# Separate the train and test data
cutoff_date = pd.Timestamp("2015-01-01")

# Train data: 2013-2014
simple_train_df = simple_df[simple_df["AGG_PERIOD_START"] < cutoff_date]
# Test data: 2015
simple_test_df  = simple_df[simple_df["AGG_PERIOD_START"] >= cutoff_date]

# Save the data
simple_train_df.to_csv("./Dataset/simple_train.csv", index=False)
simple_test_df.to_csv("./Dataset/simple_test.csv", index=False)

## Final Data Splits

In [16]:
simple_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4528737 entries, 0 to 4528736
Data columns (total 11 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   EQUIPMENTID            int64         
 1   AGG_PERIOD_START       datetime64[ns]
 2   LANE_BUNDLE_DIRECTION  string        
 3   TOTAL_VOLUME           int64         
 4   min_since_midnight     int32         
 5   simple_time_bin        int16         
 6   half_time_bin          int8          
 7   one_time_bin           int8          
 8   simple_arm             object        
 9   half_arm               object        
 10  one_arm                object        
dtypes: datetime64[ns](1), int16(1), int32(1), int64(2), int8(2), object(3), string(1)
memory usage: 311.0+ MB


In [17]:
simple_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2565698 entries, 4528737 to 7094434
Data columns (total 11 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   EQUIPMENTID            int64         
 1   AGG_PERIOD_START       datetime64[ns]
 2   LANE_BUNDLE_DIRECTION  string        
 3   TOTAL_VOLUME           int64         
 4   min_since_midnight     int32         
 5   simple_time_bin        int16         
 6   half_time_bin          int8          
 7   one_time_bin           int8          
 8   simple_arm             object        
 9   half_arm               object        
 10  one_arm                object        
dtypes: datetime64[ns](1), int16(1), int32(1), int64(2), int8(2), object(3), string(1)
memory usage: 176.2+ MB
